In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import multiprocessing  
from multiprocessing import Pool
from tqdm import tqdm

In [120]:
df = pd.read_csv("dataset_cleaned.csv")

In [121]:
docs = [(row['text'], row['DALTIX_ID']) for index, row in df.iterrows()]
len(docs)

In [124]:
%%time
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_df=0.85).fit_transform(df.text.tolist())

Wall time: 19.9 s


In [ ]:
def cal_function(i):
    cosine_similarities = linear_kernel(tfidf[i], tfidf).flatten()
    related_docs_indices = (-cosine_similarities).argsort()[:2]
    for n in related_docs_indices:
        if n == i:
            continue
        else:
            return(df['DALTIX_ID'].iat[i], df['DALTIX_ID'].iat[n], cosine_similarities[n])
            break


In [2]:
with Pool(4) as p:
    result = list(tqdm(p.imap(defs.cal_function, range(20)), total=20))

In [ ]:
pd.DataFrame(result, columns=['daltix_id_1', 'daltix_id_2', 'similarity']).to_csv("tfidf_df_ngrams.csv", index=False)